# Crypt Target Visualize Importance

In [ ]:
import os
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
from contextlib import contextmanager
from time import time
from tqdm import tqdm
import lightgbm as lgbm
import category_encoders as ce
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

# Data preparation

In [ ]:
data = pd.read_csv("../input/g-research-crypto-forecasting/train.csv")
data[0:2].T

In [ ]:
data0=data[data['Asset_ID']==0]

In [ ]:

data0['sma15T'] = data0['Count'].rolling(15).mean()/data0['Count'] -1
data0['sma60T'] = data0['Count'].rolling(60).mean()/data0['Count'] -1
data0['sma240T'] = data0['Count'].rolling(240).mean()/data0['Count'] -1
data0['return15T'] = data0['Count']/data0['Count'].shift(15) -1
data0['return60T'] = data0['Count']/data0['Count'].shift(60) -1
data0['return240T'] = data0['Count']/data0['Count'].shift(240) -1

data0['sma15P'] = data0['Open'].rolling(15).mean()/data0['Open'] -1
data0['sma60P'] = data0['Open'].rolling(60).mean()/data0['Open'] -1
data0['sma240P'] = data0['Open'].rolling(240).mean()/data0['Open'] -1
data0['return15P'] = data0['Open']/data0['Open'].shift(15) -1
data0['return60P'] = data0['Open']/data0['Open'].shift(60) -1
data0['return240P'] = data0['Open']/data0['Open'].shift(240) -1

data0['sma15H'] = data0['High'].rolling(15).mean()/data0['High'] -1
data0['sma60H'] = data0['High'].rolling(60).mean()/data0['High'] -1
data0['sma240H'] = data0['High'].rolling(240).mean()/data0['High'] -1
data0['return15H'] = data0['High']/data0['High'].shift(15) -1
data0['return60H'] = data0['High']/data0['High'].shift(60) -1
data0['return240H'] = data0['High']/data0['High'].shift(240) -1

data0['sma15L'] = data0['Low'].rolling(15).mean()/data0['Low'] -1
data0['sma60L'] = data0['Low'].rolling(60).mean()/data0['Low'] -1
data0['sma240L'] = data0['Low'].rolling(240).mean()/data0['Low'] -1
data0['return15L'] = data0['Low']/data0['Low'].shift(15) -1
data0['return60L'] = data0['Low']/data0['Low'].shift(60) -1
data0['return240L'] = data0['Low']/data0['Low'].shift(240) -1

data0['sma15C'] = data0['Close'].rolling(15).mean()/data0['Close'] -1
data0['sma60C'] = data0['Close'].rolling(60).mean()/data0['Close'] -1
data0['sma240C'] = data0['Close'].rolling(240).mean()/data0['Close'] -1
data0['return15C'] = data0['Close']/data0['Close'].shift(15) -1
data0['return60C'] = data0['Close']/data0['Close'].shift(60) -1
data0['return240C'] = data0['Close']/data0['Close'].shift(240) -1

data0['sma15V'] = data0['Volume'].rolling(15).mean()/data0['Volume'] -1
data0['sma60V'] = data0['Volume'].rolling(60).mean()/data0['Volume'] -1
data0['sma240V'] = data0['Volume'].rolling(240).mean()/data0['Volume'] -1
data0['return15V'] = data0['Volume']/data0['Volume'].shift(15) -1
data0['return60V'] = data0['Volume']/data0['Volume'].shift(60) -1
data0['return240V'] = data0['Volume']/data0['Volume'].shift(240) -1

data0['sma15W'] = data0['VWAP'].rolling(15).mean()/data0['VWAP'] -1
data0['sma60W'] = data0['VWAP'].rolling(60).mean()/data0['VWAP'] -1
data0['sma240W'] = data0['VWAP'].rolling(240).mean()/data0['VWAP'] -1
data0['return15W'] = data0['VWAP']/data0['VWAP'].shift(15) -1
data0['return60W'] = data0['VWAP']/data0['VWAP'].shift(60) -1
data0['return240W'] = data0['VWAP']/data0['VWAP'].shift(240) -1

data0['Upper_Shadow'] = data0['High'] - np.maximum(data0['Close'], data0['Open'])
data0['Lower_Shadow'] = np.minimum(data0['Close'], data0['Open']) - data0['Low']

data0['high2low'] = data0['High'] / data0['Low']
data0['close2open'] = data0['Close'] / data0['Open']
data0['high2open'] = data0['High'] / data0['Open']
data0['low2open'] = data0['Low'] / data0['Open']

data0['volume2count'] = data0['Volume'] / (data0['Count'] + 1)
data0['vwap2count'] = data0['VWAP'] / (data0['Count'] + 1)
data0['open2count'] = data0['Open'] / (data0['Count'] + 1)
data0['high2count'] = data0['High'] / (data0['Count'] + 1)
data0['low2count'] = data0['Low'] / (data0['Count'] + 1)
data0['close2count'] = data0['Close'] / (data0['Count'] + 1)

data0['count2vwap'] = data0['Count'] / (data0['VWAP'] + 1)
data0['open2vwap'] = data0['Open'] / (data0['VWAP'] + 1)
data0['high2vwap'] = data0['High'] / (data0['VWAP'] + 1)
data0['low2vwap'] = data0['Low'] / (data0['VWAP'] + 1)
data0['close2vwap'] = data0['Close'] / (data0['VWAP'] + 1)

In [ ]:
data0.info()

In [ ]:
data0=data0.dropna()

In [ ]:
df=data0.copy()
from sklearn.preprocessing import LabelEncoder
for c in df.columns:
    if df[c].dtype=='object': 
        df[c] = df[c].fillna('N')
        lbl = LabelEncoder()
        lbl.fit(list(df[c].values))
        df[c] = lbl.transform(df[c].values)

In [ ]:
data1=df
data1[0:2].T

In [ ]:
data1.columns

# Target setting

In [ ]:
target=['Target']
datay=data1[target]
datax=data1.drop([target[0]],axis=1)

In [ ]:
train=np.array(datax)
trainy=np.array(datay)
print(train.shape)
print(trainy.shape)

In [ ]:
df_columns = list(datax.columns)
print(df_columns)

In [ ]:
train_df=pd.DataFrame(train)
#test_df=pd.DataFrame(test)

In [ ]:
train_df.columns=df_columns
#test_df.columns=df_columns

In [ ]:
def create_numeric_feature(input_df):
    use_columns = df_columns 
    return input_df[use_columns].copy()

In [ ]:
from contextlib import contextmanager
from time import time

class Timer:
    def __init__(self, logger=None, format_str='{:.3f}[s]', prefix=None, suffix=None, sep=' '):

        if prefix: format_str = str(prefix) + sep + format_str
        if suffix: format_str = format_str + sep + str(suffix)
        self.format_str = format_str
        self.logger = logger
        self.start = None
        self.end = None

    @property
    def duration(self):
        if self.end is None:
            return 0
        return self.end - self.start

    def __enter__(self):
        self.start = time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time()
        out_str = self.format_str.format(self.duration)
        if self.logger:
            self.logger.info(out_str)
        else:
            print(out_str)

In [ ]:
from tqdm import tqdm

def to_feature(input_df):

    processors = [
        create_numeric_feature,
    ]
    
    out_df = pd.DataFrame()
    
    for func in tqdm(processors, total=len(processors)):
        with Timer(prefix='create' + func.__name__ + ' '):
            _df = func(input_df)

        assert len(_df) == len(input_df), func.__name__
        out_df = pd.concat([out_df, _df], axis=1)
        
    return out_df

In [ ]:
train_feat_df = to_feature(train_df)
#test_feat_df = to_feature(test_df)

# Model

In [ ]:
import lightgbm as lgbm
from sklearn.metrics import mean_squared_error

def fit_lgbm(X, y, cv, 
             params: dict=None, 
             verbose: int=50):

    if params is None:
        params = {}

    models = []
    oof_pred = np.zeros_like(y, dtype=np.float)

    for i, (idx_train, idx_valid) in enumerate(cv): 
        x_train, y_train = X[idx_train], y[idx_train]
        x_valid, y_valid = X[idx_valid], y[idx_valid]

        clf = lgbm.LGBMRegressor(**params)
        
        with Timer(prefix='fit fold={} '.format(i)):
            clf.fit(x_train, y_train, 
                    eval_set=[(x_valid, y_valid)],  
                    early_stopping_rounds=100,
                    verbose=verbose)

        pred_i = clf.predict(x_valid)
        oof_pred[idx_valid] = pred_i
        models.append(clf)
        print(f'Fold {i} RMSLE: {mean_squared_error(y_valid, pred_i) ** .5:.4f}')
        print()

    score = mean_squared_error(y, oof_pred) ** .5
    print('-' * 50)
    print('FINISHED | Whole RMSLE: {:.4f}'.format(score))
    return oof_pred, models

In [ ]:
params = {
    'objective': 'rmse', 
    'learning_rate': .1,
    'reg_lambda': 1.,
    'reg_alpha': .1,
    'max_depth': 5, 
    'n_estimators': 10000, 
    'colsample_bytree': .5, 
    'min_child_samples': 10,
    'subsample_freq': 3,
    'subsample': .9,
    'importance_type': 'gain', 
    'random_state': 71,
    'num_leaves': 62
}

In [ ]:
y = trainy
ydf=pd.DataFrame(y)
ydf

In [ ]:
from sklearn.model_selection import KFold

for i in range(1):
    fold = KFold(n_splits=5, shuffle=True, random_state=71)
    ydfi=ydf.iloc[:,i]
    y=np.array(ydfi)
    cv = list(fold.split(train_feat_df, y))
    oof, models = fit_lgbm(train_feat_df.values, y, cv, params=params, verbose=500)
    
    fig,ax = plt.subplots(figsize=(6,6))
    ax.set_title(target[i],fontsize=20)
    ax.set_xlabel('Predicted Target',fontsize=12)
    ax.set_ylabel('Actual Target',fontsize=12)
    ax.scatter(oof,y)


In [ ]:
print(target)

# Visualize Importance

In [ ]:
def visualize_importance(models, feat_train_df):

    feature_importance_df = pd.DataFrame()
    for i, model in enumerate(models):
        _df = pd.DataFrame()
        _df['feature_importance'] = model.feature_importances_
        _df['column'] = feat_train_df.columns
        _df['fold'] = i + 1
        feature_importance_df = pd.concat([feature_importance_df, _df], 
                                          axis=0, ignore_index=True)

    order = feature_importance_df.groupby('column')\
        .sum()[['feature_importance']]\
        .sort_values('feature_importance', ascending=False).index[:50]

    fig, ax = plt.subplots(figsize=(8, max(6, len(order) * .25)))
    sns.boxenplot(data=feature_importance_df, 
                  x='feature_importance', 
                  y='column', 
                  order=order, 
                  ax=ax, 
                  palette='viridis', 
                  orient='h')
    
    ax.tick_params(axis='x', rotation=0)
    #ax.set_title('Importance')
    ax.grid()
    fig.tight_layout()
    
    return fig,ax

#fig, ax = visualize_importance(models, train_feat_df)

In [ ]:
for i in range(1):
    fold = KFold(n_splits=5, shuffle=True, random_state=71)
    ydfi=ydf.iloc[:,i]
    y=np.array(ydfi)
    cv = list(fold.split(train_feat_df, y))
    oof, models = fit_lgbm(train_feat_df.values, y, cv, params=params, verbose=500)
    fig, ax = visualize_importance(models, train_feat_df)
    ax.set_title(target[i]+' Imortance',fontsize=20)

In [ ]:
fig,ax = plt.subplots(figsize=(6,6))
ax.set_title('Low vs Target',fontsize=20)
ax.set_xlabel('Low',fontsize=12)
ax.set_ylabel('Target',fontsize=12)
ax.scatter(data0['Low'],data0['Target'])

In [ ]:
fig,ax = plt.subplots(figsize=(6,6))
ax.set_title('Close vs Target',fontsize=20)
ax.set_xlabel('Close',fontsize=12)
ax.set_ylabel('Target',fontsize=12)
ax.scatter(data0['Close'],data0['Target'])